# Youkea Cost Accounting Analysis - Solution
## Activity-Based Costing (ABC) Analysis

This notebook provides a complete solution to the Youkea cost accounting case study, including:
1. Order entry cost calculations
2. Warehouse and delivery cost analysis
3. Customer profitability analysis
4. Visualization of results

Each section includes test validation using the cost_accounting_tests module.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.testing.cost_accounting_tests import (
    check_activity_costs,
    check_customer_profitability,
    check_cost_allocation
)

## Step 1: Calculate Order Entry Cost Rates

First, we'll calculate the cost rates for phone and web orders using the given data:
- Phone orders: 85% of total order entry costs
- Web orders: 15% of total order entry costs
- Total order entry expenses: $20,000

In [ ]:
# Order entry cost analysis
total_order_entry_cost = 20000
phone_order_cost = total_order_entry_cost * 0.85
web_order_cost = total_order_entry_cost * 0.15

# Calculate number of orders by type
total_orders = 700
phone_orders = total_orders * 0.20  # 20% are phone orders
web_orders = total_orders * 0.80    # 80% are web orders
phone_order_lines = 800             # Given in problem

# Calculate cost per order type
cost_per_phone_order_line = phone_order_cost / phone_order_lines
cost_per_web_order = web_order_cost / web_orders

print(f"Cost Analysis for Order Entry:")
print(f"Cost per phone order line: ${cost_per_phone_order_line:.2f}")
print(f"Cost per web order: ${cost_per_web_order:.2f}")

# Test the activity costs
activity_costs = {
    'order_entry': total_order_entry_cost,
    'warehouse_handling': 440000,  # Warehouse personnel + other expenses
    'delivery': 110000,           # Delivery personnel + other expenses
    'freight': 350000             # Common carrier costs
}

total_cost = 920000  # Sum of all operational costs

if check_activity_costs(activity_costs, total_cost):
    print("✓ Activity costs correctly calculated!")
else:
    print("✗ Check your activity cost calculations")

## Step 2: Calculate Delivery and Freight Costs

Now we'll analyze the delivery and freight costs:
- Common carrier: Cost based on number of chairs
- Express delivery: Cost based on number of shipments
- Total chairs: 4,800
- Express deliveries: 100 shipments (800 chairs)

In [ ]:
# Delivery cost analysis
total_chairs = 4800
express_shipments = 100
express_chairs = 800
regular_chairs = total_chairs - express_chairs

# Calculate costs
total_freight_cost = 350000
total_delivery_cost = 110000

# Cost per chair for regular shipping
cost_per_regular_chair = total_freight_cost / regular_chairs

# Cost per express shipment
cost_per_express_shipment = total_delivery_cost / express_shipments

print(f"Delivery Cost Analysis:")
print(f"Cost per regular shipped chair: ${cost_per_regular_chair:.2f}")
print(f"Cost per express shipment: ${cost_per_express_shipment:.2f}")

# Test cost allocation
cost_drivers = {
    'regular_shipping': regular_chairs,
    'express_shipping': express_shipments
}

activities = {
    'regular_shipping': {'rate': cost_per_regular_chair},
    'express_shipping': {'rate': cost_per_express_shipment}
}

total_shipping_cost = total_freight_cost + total_delivery_cost

if check_cost_allocation(total_shipping_cost, cost_drivers, activities):
    print("✓ Shipping cost allocation correct!")
else:
    print("✗ Check your shipping cost calculations")

## Step 3: Customer Profitability Analysis

Let's analyze profitability for two example customers with different order patterns:

In [ ]:
def calculate_customer_cost(chairs, web_orders, phone_order_lines, express_deliveries):
    """Calculate total cost for a customer based on their order pattern.
    
    Args:
        chairs: Number of chairs ordered
        web_orders: Number of web orders
        phone_order_lines: Number of phone order lines
        express_deliveries: Number of express deliveries
    
    Returns:
        float: Total cost for the customer
    """
    # Order entry costs
    order_entry_cost = (web_orders * cost_per_web_order + 
                       phone_order_lines * cost_per_phone_order_line)
    
    # Shipping costs
    regular_chairs = chairs - (express_deliveries * 8)  # Assume 8 chairs per express delivery
    shipping_cost = (regular_chairs * cost_per_regular_chair +
                    express_deliveries * cost_per_express_shipment)
    
    return order_entry_cost + shipping_cost

# Example customers
customers = {
    'Customer A': {
        'chairs': 100,
        'web_orders': 10,
        'phone_lines': 5,
        'express': 2,
        'revenue': 50000
    },
    'Customer B': {
        'chairs': 100,
        'web_orders': 5,
        'phone_lines': 15,
        'express': 8,
        'revenue': 50000
    }
}

# Calculate costs and profitability
revenue = {}
costs = {}

for customer, data in customers.items():
    revenue[customer] = data['revenue']
    costs[customer] = calculate_customer_cost(
        data['chairs'],
        data['web_orders'],
        data['phone_lines'],
        data['express']
    )
    
    profit = revenue[customer] - costs[customer]
    print(f"\n{customer}:")
    print(f"Revenue: ${revenue[customer]:,.2f}")
    print(f"Costs: ${costs[customer]:,.2f}")
    print(f"Profit: ${profit:,.2f}")

# Test profitability calculations
if check_customer_profitability(revenue, costs):
    print("\n✓ Customer profitability analysis correct!")
else:
    print("\n✗ Check your profitability calculations")

## Step 4: Visualization and Analysis

Let's visualize the cost breakdown and profitability comparison:

In [ ]:
# Create cost breakdown visualization
plt.figure(figsize=(12, 6))

# Cost components for each customer
customers_list = list(customers.keys())
cost_components = ['Order Entry', 'Shipping']

# Calculate cost components
order_entry_costs = []
shipping_costs = []

for customer, data in customers.items():
    # Order entry
    oe_cost = (data['web_orders'] * cost_per_web_order +
               data['phone_lines'] * cost_per_phone_order_line)
    order_entry_costs.append(oe_cost)
    
    # Shipping
    regular_chairs = data['chairs'] - (data['express'] * 8)
    ship_cost = (regular_chairs * cost_per_regular_chair +
                 data['express'] * cost_per_express_shipment)
    shipping_costs.append(ship_cost)

# Create stacked bar chart
width = 0.35
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(customers_list, order_entry_costs, width, label='Order Entry')
ax.bar(customers_list, shipping_costs, width, bottom=order_entry_costs, label='Shipping')

ax.set_ylabel('Cost ($)')
ax.set_title('Cost Breakdown by Customer')
ax.legend()

plt.tight_layout()
plt.show()

# Analysis summary
print("\nKey Findings:")
print("1. Customer A (primarily web orders) has lower order entry costs")
print("2. Customer B (more express deliveries) has higher shipping costs")
print("3. The difference in profitability is mainly driven by order processing method")